In [1]:
import cv2
import time
import skimage
import numpy as np
from matplotlib import pyplot as plt
from skimage.feature import ORB
import match_orb
import random

In [2]:
img1_raw = cv2.imread('hopkins1.jpg')
img2_raw = cv2.imread('hopkins2.jpg')

img1 = cv2.cvtColor(img1_raw, cv2.COLOR_RGB2GRAY)
img2 = cv2.cvtColor(img2_raw, cv2.COLOR_RGB2GRAY)

detector_extractor1 = ORB(n_keypoints=500)
detector_extractor2 = ORB(n_keypoints=500)

detector_extractor1.detect_and_extract(img1)
detector_extractor2.detect_and_extract(img2)

In [3]:
def displaying_matches(image1, image2, feature_coords1, feature_coords2, matches):
    if image1.shape[0] == image2.shape[0]:
        SbS = np.concatenate((image1,image2), axis=1)
    elif image1.shape[0] > image2.shape[0]:
        image2_extended = cv2.copyMakeBorder(image2,0,image1.shape[0]-image2.shape[0],0,0,cv2.BORDER_CONSTANT)
        SbS = np.concatenate((image1,image2_extended), axis=1)
    else:
        image1_extended = cv2.copyMakeBorder(image1,0,image2.shape[0]-image1.shape[0],0,0,cv2.BORDER_CONSTANT)
        SbS = np.concatenate((image1_extended,image2), axis=1)
    img1_width = image1.shape[1]
    color = np.array([255,0,0])
    for i in range(len(matches)):
        cv2.line(SbS, (int(feature_coords1[matches[i][0]][1]), int(feature_coords1[matches[i][0]][0])), \
            (int(feature_coords2[matches[i][1]][1])+img1_width, int(feature_coords2[matches[i][1]][0])), color)
    for k in range(len(feature_coords1)):
        SbS[int(feature_coords1[k][0]), int(feature_coords1[k][1])] = color
    for k in range(len(feature_coords2)):
        SbS[int(feature_coords2[k][0]), int(feature_coords2[k][1])+img1_width] = color
    cv2.namedWindow("match features in two images", cv2.WINDOW_NORMAL)
    cv2.imshow('match features in two images', SbS)
    cv2.waitKey(0)

    return SbS

In [4]:
reload(match_orb)
matches = match_orb.match_orb(detector_extractor1.descriptors, detector_extractor2.descriptors)
SbS = displaying_matches(img1_raw, img2_raw, detector_extractor1.keypoints, detector_extractor2.keypoints, matches)

match_orb() started.
match_orb() finished. Running time:  55.1610000134


In [5]:
def compute_proj_xform(matches,features1,features2,image1,image2,iter_rounds = 100):

    print "compute_proj_xform() started."
    start_time = time.time()
    
    proj_xform = np.zeros((3,3))
    height, width = image1.shape[0:2]
    image_raw1 = np.copy(image1)
    image_raw2 = np.copy(image2)
    image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
    image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)

    # RANSAC select points for computing xform
    best_xform = np.zeros((3,3))
    best_xform[2,2] = 1
    best_inlier_std = float('Inf')
    best_inlier_count = 0
    best_inlier_index = []
    inlier_thresh = 5
    curr_xform = np.zeros((3,3))

    for i in range(iter_rounds):
        inliers_count = 0
        inliers_dist = []
        inliers_index = []

        # Randomly choose 4 matches
        rand_match_index = np.random.choice(range(len(matches)),4)
        rand_match = []
        for k in range(4):
            rand_match.append(matches[rand_match_index[k]])

        # Check whether they are good points: not too close, not colinear
        x1a = features1[rand_match[0][0]][1]
        y1a = features1[rand_match[0][0]][0]
        x2a = features1[rand_match[1][0]][1]
        y2a = features1[rand_match[1][0]][0]
        x3a = features1[rand_match[2][0]][1]
        y3a = features1[rand_match[2][0]][0]
        x4a = features1[rand_match[3][0]][1]
        y4a = features1[rand_match[3][0]][0]
        x_mean = np.mean([x1a,x2a,x3a,x4a])
        y_mean = np.mean([y1a,y2a,y3a,y4a])
        xyz1 = np.array([[x1a-x_mean,y1a-y_mean,1],[x2a-x_mean,y2a-y_mean,1],[x3a-x_mean,y3a-y_mean,1]])
        xyz2 = np.array([[x4a-x_mean,y4a-y_mean,1],[x2a-x_mean,y2a-y_mean,1],[x3a-x_mean,y3a-y_mean,1]])
        xyz3 = np.array([[x1a-x_mean,y1a-y_mean,1],[x4a-x_mean,y4a-y_mean,1],[x3a-x_mean,y3a-y_mean,1]])
        xyz4 = np.array([[x1a-x_mean,y1a-y_mean,1],[x2a-x_mean,y2a-y_mean,1],[x4a-x_mean,y4a-y_mean,1]])
        if np.absolute(np.linalg.det(xyz1)) < 100 or np.absolute(np.linalg.det(xyz2)) < 100 or \
            np.absolute(np.linalg.det(xyz3)) < 100 or np.absolute(np.linalg.det(xyz4)) < 100:
                continue
        x1b = features2[rand_match[0][1]][1]
        y1b = features2[rand_match[0][1]][0]
        x2b = features2[rand_match[1][1]][1]
        y2b = features2[rand_match[1][1]][0]
        x3b = features2[rand_match[2][1]][1]
        y3b = features2[rand_match[2][1]][0]
        x4b = features2[rand_match[3][1]][1]
        y4b = features2[rand_match[3][1]][0]
        x_mean = np.mean([x1b,x2b,x3b,x4b])
        y_mean = np.mean([y1b,y2b,y3b,y4b])
        xyz1 = np.array([[x1b-x_mean,y1b-y_mean,1],[x2b-x_mean,y2b-y_mean,1],[x3b-x_mean,y3b-y_mean,1]])
        xyz2 = np.array([[x4b-x_mean,y4b-y_mean,1],[x2b-x_mean,y2b-y_mean,1],[x3b-x_mean,y3b-y_mean,1]])
        xyz3 = np.array([[x1b-x_mean,y1b-y_mean,1],[x4b-x_mean,y4b-y_mean,1],[x3b-x_mean,y3b-y_mean,1]])
        xyz4 = np.array([[x1b-x_mean,y1b-y_mean,1],[x2b-x_mean,y2b-y_mean,1],[x4b-x_mean,y4b-y_mean,1]])
        if np.absolute(np.linalg.det(xyz1)) < 100 or np.absolute(np.linalg.det(xyz2)) < 100 or \
            np.absolute(np.linalg.det(xyz3)) < 100 or np.absolute(np.linalg.det(xyz4)) < 100:
                continue

        # Compute curr_xform based on the 3 points chosen
        A = np.array([[x1a,y1a,1,0,0,0,-x1b*x1a,-x1b*y1a,-x1b],\
            [0,0,0,x1a,y1a,1,-y1b*x1a,-y1b*y1a,-y1b],\
            [x2a,y2a,1,0,0,0,-x2b*x2a,-x2b*y2a,-x2b],\
            [0,0,0,x2a,y2a,1,-y2b*x2a,-y2b*y2a,-y2b],\
            [x3a,y3a,1,0,0,0,-x3b*x3a,-x3b*y3a,-x3b],\
            [0,0,0,x3a,y3a,1,-y3b*x3a,-y3b*y3a,-y3b],\
            [x4a,y4a,1,0,0,0,-x4b*x4a,-x4b*y4a,-x4b],\
            [0,0,0,x4a,y4a,1,-y4b*x4a,-y4b*y4a,-y4b]]);
        AT = np.transpose(A)
        ATA = np.dot(AT,A)
        U,D,UT = np.linalg.svd(ATA)
        curr_xform = np.reshape(U[:,8],(3,3))

        # Calculate the distance for each feature under curr_xform and count inliers
        for j in range(len(matches)):
            pt1 = np.array([[features1[matches[j][0]][1]],[features1[matches[j][0]][0]],[1]])
            pt2 = np.array([[features2[matches[j][1]][1]],[features2[matches[j][1]][0]],[1]])
            pt1_xform = np.dot(curr_xform,pt1)/(curr_xform[2,0]*pt1[0]+curr_xform[2,1]*pt1[1]+curr_xform[2,2])
            dist = np.sqrt((pt1_xform[0]-pt2[0])**2 + (pt1_xform[1]-pt2[1])**2)
            if dist < inlier_thresh:
                inliers_count = inliers_count + 1
                inliers_dist.append(dist)
                inliers_index.append(j)

        # Update the best transform
        if inliers_count > best_inlier_count:
            best_inlier_count = inliers_count
            best_xform = np.copy(curr_xform)
            best_inlier_index = inliers_index
        else:
            if inliers_count == best_inlier_count and np.std(inliers_dist) < best_inlier_std:
                best_inlier_std = np.std(inliers_dist)
                best_xform = np.copy(curr_xform)
                best_inlier_index = inliers_index

    print "found good matching pairs:", best_inlier_count
    # Show the RANSAC works
    if image_raw1.shape[0] == image_raw2.shape[0]:
        SbS = np.concatenate((image_raw1,image_raw2), axis=1)
    elif image_raw1.shape[0] > image_raw2.shape[0]:
        image2_extended = cv2.copyMakeBorder(image_raw2,0,image_raw1.shape[0]-image_raw2.shape[0],0,0,cv2.BORDER_CONSTANT)
        SbS = np.concatenate((image_raw1,image2_extended), axis=1)
    else:
        image1_extended = cv2.copyMakeBorder(image_raw1,0,image_raw2.shape[0]-image_raw1.shape[0],0,0,cv2.BORDER_CONSTANT)
        SbS = np.concatenate((image1_extended,image_raw2), axis=1)
    color_outlier = np.array([0,0,255])
    color_inlier = np.array([0,255,0])
    for i in range(len(matches)):
        if i in best_inlier_index:
            cv2.line(SbS, (int(features1[matches[i][0]][1]), int(features1[matches[i][0]][0])), \
                     (int(features2[matches[i][1]][1])+width, int(features2[matches[i][1]][0])), color_inlier)
        else:
            cv2.line(SbS, (int(features1[matches[i][0]][1]), int(features1[matches[i][0]][0])), \
                     (int(features2[matches[i][1]][1])+width, int(features2[matches[i][1]][0])), color_outlier)
    
    elapsed_time = time.time() - start_time
    cv2.namedWindow("RANSAC", cv2.WINDOW_NORMAL)
    cv2.imshow('RANSAC', SbS)
    cv2.waitKey(0)

    proj_xform = best_xform

    print "compute_proj_xform() finished. Running time: ", elapsed_time

    return proj_xform, best_inlier_index

In [6]:
proj_xform, ransac_inliers = compute_proj_xform(matches,detector_extractor1.keypoints,detector_extractor2.keypoints,img1_raw,img2_raw,iter_rounds = 500)
# img1_raw, img2_raw, detector_extractor1.keypoints, detector_extractor2.keypoints, matches

compute_proj_xform() started.
found good matching pairs: 28
compute_proj_xform() finished. Running time:  11.0080001354


In [7]:
good_matches = []
for i in range(len(matches)):
        if i in ransac_inliers:
            good_matches.append(matches[i])
print len(good_matches)

28


In [14]:
print good_matches

[[154, 87], [466, 103], [62, 148], [195, 164], [102, 172], [233, 176], [408, 199], [152, 206], [156, 236], [207, 241], [112, 269], [108, 287], [400, 294], [141, 310], [490, 316], [149, 319], [360, 320], [413, 328], [276, 347], [312, 350], [179, 371], [192, 393], [495, 397], [328, 414], [243, 420], [460, 444], [345, 480], [242, 485]]


In [66]:
def compute_fundamental(features1, features2, matches):
    if len(matches) < 9:
        print "Not enough matches to compute fundamental matrix!"
        return np.zeros([3,3])
    else:
        A = np.ones([len(matches),9])
        for i in range(len(matches)):
            ul = int(features1[matches[i][0]][1])
            vl = int(features1[matches[i][0]][0])
            ur = int(features2[matches[i][1]][1])
            vr = int(features2[matches[i][1]][0])
            A[i,0] = ul*ur
            A[i,1] = ul*vr
            A[i,2] = ul
            A[i,3] = vl*ur
            A[i,4] = vl*vr
            A[i,5] = vl
            A[i,6] = ur
            A[i,7] = vr
        AT = np.transpose(A)
        ATA = np.dot(AT,A)
        U,D,UT = np.linalg.svd(ATA)
        fundamental = np.reshape(U[:,8],(3,3))
        for i in range(len(matches)):
            left_coords = np.array([int(features1[matches[i][0]][1]),int(features1[matches[i][0]][0]),1])
            right_coords = np.array([int(features2[matches[i][1]][1]),int(features2[matches[i][1]][0]),1])
            res = np.dot(np.dot(left_coords, fundamental), right_coords)
            print "residual: ", res
        return fundamental

In [67]:
F = compute_fundamental(detector_extractor1.keypoints, detector_extractor2.keypoints, good_matches)
print F
# F2 = compute_fundamental(detector_extractor1.keypoints, detector_extractor2.keypoints, good_matches[0:20])
# print F2

residual:  0.000116928656694
residual:  7.68860096855e-05
residual:  -0.000988336057641
residual:  -0.000277909685233
residual:  0.000489186288813
residual:  -0.000565976940844
residual:  0.000180742711634
residual:  -0.000315209260202
residual:  0.0001537320032
residual:  -0.000176755006334
residual:  -0.000429955075229
residual:  0.000589257935578
residual:  -8.72962098324e-05
residual:  2.1641266995e-05
residual:  -0.000169237374815
residual:  -0.000377964093125
residual:  6.16412512921e-05
residual:  -0.00032705959042
residual:  0.00029835706475
residual:  0.000536391180781
residual:  0.000397808240845
residual:  0.000430574758811
residual:  -0.000450667532878
residual:  -0.000148125518475
residual:  0.00042047587245
residual:  0.000243736126492
residual:  0.00122072483647
residual:  -0.000917435908967
[[  2.37594412e-07   1.21793584e-05  -3.56497365e-03]
 [ -1.01239041e-05  -2.02154158e-08   1.34337856e-03]
 [  2.75604900e-03  -4.70232974e-03   9.99977889e-01]]


In [69]:
def draw_epipolar(F, img1, img2, features1, features2, matches, num_points = 2):
    # num_point should not be larger than 5
    height, width = img1.shape[0:2]
    if img1.shape[0] == img2.shape[0]:
        SbS = np.concatenate((img1,img2), axis=1)
    elif img1.shape[0] > img2.shape[0]:
        image2_extended = cv2.copyMakeBorder(img2,0,img1.shape[0]-img2.shape[0],0,0,cv2.BORDER_CONSTANT)
        SbS = np.concatenate((img1,image2_extended), axis=1)
    else:
        image1_extended = cv2.copyMakeBorder(img1,0,img2.shape[0]-img1.shape[0],0,0,cv2.BORDER_CONSTANT)
        SbS = np.concatenate((image1_extended,img2), axis=1)
    
    color_change = 0
    match_sample = random.sample(matches, num_points)
    for i in range(num_points):
        match = match_sample[i]
        left_coords = np.array([int(features1[match[0]][1]),int(features1[match[0]][0]),1])
        right_coords = np.array([int(features2[match[1]][1]),int(features2[match[1]][0]),1])
        cv2.circle(SbS, (left_coords[0], left_coords[1]), 10, np.array([color_change,255-color_change,color_change]), 2)
        cv2.circle(SbS, (right_coords[0]+width, right_coords[1]), 10, np.array([color_change,255-color_change,color_change]), 2)
        print left_coords
        print right_coords
        print F
        params = np.dot(left_coords, F)
        print params
        error = np.dot(params, right_coords)
        print "LSQ error:", error
        line_ends = []
        if -params[2]/params[1] >= 0 and -params[2]/params[1] <= height:
            line_ends.append([0, int(-params[2]/params[1])])
        if -(params[0]*width+params[2])/params[1] >= 0 and -(params[0]*width+params[2])/params[1] <= height:
            line_ends.append([width, int(-(params[0]*width+params[2])/params[1])])
        if -params[2]/params[0] >= 0 and -params[2]/params[0] <= width:
            line_ends.append([int(-params[2]/params[0]), 0])
        if -(params[1]*height+params[2])/params[0] >= 0 and -(params[1]*height+params[2])/params[0] <= width:
            line_ends.append([int(-(params[1]*height+params[2])/params[0]), height])
        if len(line_ends) == 2:
            print "line_ends:", line_ends
            cv2.line(SbS, (line_ends[0][0]+width,line_ends[0][1]), (line_ends[1][0]+width, line_ends[1][1]), np.array([color_change,255-color_change,color_change]), 2)
        color_change += 50
        
    cv2.namedWindow("Epipolar", cv2.WINDOW_NORMAL)
    cv2.imshow('Epipolar', SbS)
    cv2.waitKey(0)
        

In [71]:
draw_epipolar(F, img1_raw, img2_raw, detector_extractor1.keypoints, detector_extractor2.keypoints, good_matches, 4)
# img1_raw, img2_raw, detector_extractor1.keypoints, detector_extractor2.keypoints, matches

[383 312   1]
[128 319   1]
[[  2.37594412e-07   1.21793584e-05  -3.56497365e-03]
 [ -1.01239041e-05  -2.02154158e-08   1.34337856e-03]
 [  2.75604900e-03  -4.70232974e-03   9.99977889e-01]]
[ -3.11610421e-04  -4.39426727e-05   5.37270914e-02]
LSQ error: -0.000176755006334
line_ends: [[172, 0], [64, 768L]]
[381 311   1]
[126 319   1]
[[  2.37594412e-07   1.21793584e-05  -3.56497365e-03]
 [ -1.01239041e-05  -2.02154158e-08   1.34337856e-03]
 [  2.75604900e-03  -4.70232974e-03   9.99977889e-01]]
[ -3.01961705e-04  -6.82811741e-05   5.95136602e-02]
LSQ error: -0.000315209260202
line_ends: [[197, 0], [23, 768L]]
[373 174   1]
[117 187   1]
[[  2.37594412e-07   1.21793584e-05  -3.56497365e-03]
 [ -1.01239041e-05  -2.02154158e-08   1.34337856e-03]
 [  2.75604900e-03  -4.70232974e-03   9.99977889e-01]]
[ 0.00108311 -0.00016295 -0.09600941]
LSQ error: 0.000243736126492
line_ends: [[88, 0], [204, 768L]]
[363 231   1]
[105 242   1]
[[  2.37594412e-07   1.21793584e-05  -3.56497365e-03]
 [ -1.0123

In [48]:
print F2

[[  2.85023830e-06   3.85413147e-06  -2.88401156e-03]
 [ -1.97791500e-07  -5.61057548e-07   3.08822606e-04]
 [ -1.02287227e-03  -1.29788964e-03   9.99994428e-01]]


In [14]:
aaa = [1,2,3,4]
bbb = random.sample(aaa,2)
print bbb

[3, 1]


In [40]:
x = np.array([1,2,1])
Te = np.array([[1,2,3],[4,5,6],[7,8,9]])
print Te, np.dot(x, Te)

[[1 2 3]
 [4 5 6]
 [7 8 9]] [16 20 24]


In [38]:
print -7.32775991e-04 -1.48506292e-03 +9.99994710e-01

0.997776871089


In [54]:
a = [1,2,3,4]
b = [11,23,33,44]
total = [a,b]
with open("testsave.txt", "wb") as fp:   #Pickling
    pickle.dump(total, fp)

In [55]:
with open("testsave.txt", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
print b

[[1, 2, 3, 4], [11, 23, 33, 44]]


In [49]:
img1 = Image.open("./facedeeplearning/lfw/Abdullah/Abdullah_0001.jpg")

translate_x = int(10 - random.random()*20)
translate_y = int(10 - random.random()*20)
img1 = img1.transform(img1.size, Image.AFFINE, (1,0,translate_x,0,1,translate_y))
img1.save('test23.bmp')

# angle = 30 - random.random()*60
# img1 = img1.rotate(angle)
# img1.show()

# img1 = img1.transpose(Image.FLIP_LEFT_RIGHT)

# ratio = 0.6*random.random() + 0.7
# if ratio > 1:
#     old_size = img1.size
#     new_size = tuple([int(i*ratio) for i in img1.size])
#     img1 = img1.resize(new_size, Image.ANTIALIAS)
#     left = abs((old_size[0] - new_size[0])/2)
#     top = abs((old_size[1] - new_size[1])/2)
#     right = abs((old_size[0] + new_size[0])/2)
#     bottom = abs((old_size[1] + new_size[1])/2)
#     img1 = img1.crop((left,top,right,bottom))
# else:
#     old_size = img1.size
#     new_size = tuple([int(i*ratio) for i in img1.size])
#     img1 = img1.resize(new_size, Image.ANTIALIAS)
#     left = 0
#     top = 0
#     right = old_size[0]
#     bottom = old_size[1]
#     img1 = img1.crop((left,top,right,bottom))


print img1.size

(250, 250)


In [35]:
# print tuple([i*1.2 for i in img1.size]) - (250, 250)
print img1.size
# img1.show()

AttributeError: 'NoneType' object has no attribute 'size'

In [2]:
def displaying_matches(image1, image2, feature_coords1, feature_coords2, matches):
    if image1.shape[0] == image2.shape[0]:
        SbS = np.concatenate((image1,image2), axis=1)
    elif image1.shape[0] > image2.shape[0]:
        image2_extended = cv2.copyMakeBorder(image2,0,image1.shape[0]-image2.shape[0],0,0,cv2.BORDER_CONSTANT)
        SbS = np.concatenate((image1,image2_extended), axis=1)
    else:
        image1_extended = cv2.copyMakeBorder(image1,0,image2.shape[0]-image1.shape[0],0,0,cv2.BORDER_CONSTANT)
        SbS = np.concatenate((image1_extended,image2), axis=1)
    img1_width = image1.shape[1]
    color = np.array([255,0,0])
    for i in range(len(matches)):
        cv2.line(SbS, (int(feature_coords1[matches[i][0]][1]), int(feature_coords1[matches[i][0]][0])), \
            (int(feature_coords2[matches[i][1]][1])+img1_width, int(feature_coords2[matches[i][1]][0])), color)
    for k in range(len(feature_coords1)):
        SbS[int(feature_coords1[k][0]), int(feature_coords1[k][1])] = color
    for k in range(len(feature_coords2)):
        SbS[int(feature_coords2[k][0]), int(feature_coords2[k][1])+img1_width] = color
    cv2.namedWindow("match features in two images", cv2.WINDOW_NORMAL)
    cv2.imshow('match features in two images', SbS)
    cv2.waitKey(0)

    return SbS

In [40]:
img1_raw = cv2.imread('hopkins1.JPG')
img2_raw = cv2.imread('hopkins2.JPG')
img1 = cv2.cvtColor(img1_raw, cv2.COLOR_RGB2GRAY)
img2 = cv2.cvtColor(img2_raw, cv2.COLOR_RGB2GRAY)
detector_extractor1 = ORB(n_keypoints=300)
detector_extractor2 = ORB(n_keypoints=300)
detector_extractor1.detect_and_extract(img1)
detector_extractor2.detect_and_extract(img2)
# matches = match_descriptors(detector_extractor1.descriptors, detector_extractor2.descriptors)

In [43]:
matches1 = match_descriptors(detector_extractor1.descriptors, detector_extractor2.descriptors)

In [41]:
reload(match_orb)
matches2 = match_orb.match_orb(detector_extractor1.descriptors, detector_extractor2.descriptors)

match_orb() started.
match_orb() finished. Running time:  21.5709998608


In [35]:
print matches2

[[401, 4], [693, 7], [608, 11], [262, 18], [518, 22], [698, 30], [221, 33], [712, 39], [213, 44], [864, 46], [506, 53], [923, 57], [834, 58], [730, 59], [313, 61], [417, 62], [43, 64], [458, 65], [520, 69], [299, 70], [804, 75], [533, 79], [87, 80], [154, 87], [78, 90], [329, 95], [392, 96], [979, 101], [579, 106], [4, 107], [318, 111], [703, 124], [187, 131], [416, 132], [67, 134], [398, 143], [330, 146], [62, 148], [308, 153], [794, 155], [36, 158], [838, 161], [195, 164], [101, 170], [536, 171], [102, 172], [233, 176], [374, 178], [138, 183], [790, 185], [813, 186], [219, 188], [377, 195], [293, 196], [839, 197], [408, 199], [912, 201], [83, 203], [232, 205], [152, 206], [225, 208], [126, 221], [547, 224], [103, 226], [862, 228], [18, 230], [801, 235], [156, 236], [207, 241], [987, 242], [405, 243], [274, 246], [566, 248], [723, 251], [614, 254], [473, 259], [756, 260], [954, 264], [112, 269], [123, 276], [502, 278], [647, 285], [108, 287], [358, 290], [174, 291], [205, 293], [400, 

In [22]:
print sum([i**2 for i in [int(m)-int(n) for m,n in zip(detector_extractor1.descriptors[1],detector_extractor2.descriptors[1])]])

118


In [44]:
displaying_matches(img1_raw, img2_raw, detector_extractor1.keypoints, detector_extractor2.keypoints, matches1)

array([[[40, 65, 75],
        [40, 65, 75],
        [41, 66, 76],
        ..., 
        [18, 27, 30],
        [15, 27, 29],
        [14, 26, 28]],

       [[39, 65, 77],
        [39, 65, 77],
        [40, 66, 78],
        ..., 
        [17, 26, 29],
        [14, 26, 28],
        [13, 25, 27]],

       [[38, 66, 77],
        [40, 66, 78],
        [41, 67, 79],
        ..., 
        [16, 25, 28],
        [14, 26, 28],
        [13, 25, 27]],

       ..., 
       [[15, 26, 30],
        [15, 26, 30],
        [16, 27, 31],
        ..., 
        [ 4,  7,  5],
        [ 4,  7,  5],
        [ 4,  7,  5]],

       [[18, 29, 33],
        [17, 28, 32],
        [17, 28, 32],
        ..., 
        [ 6,  9,  7],
        [ 6,  9,  7],
        [ 3,  8,  6]],

       [[19, 30, 34],
        [19, 30, 34],
        [18, 29, 33],
        ..., 
        [ 7, 10,  8],
        [ 6,  9,  7],
        [ 4,  9,  7]]], dtype=uint8)

In [34]:
img1 = np.zeros((100, 100))
img2 = np.zeros_like(img1)
np.random.seed(1)
square = np.random.rand(20, 20)
img1[40:60, 40:60] = square
img2[53:73, 53:73] = square
detector_extractor1 = ORB(n_keypoints=10)
detector_extractor2 = ORB(n_keypoints=10)
detector_extractor1.detect_and_extract(img1)
detector_extractor2.detect_and_extract(img2)
matches = match_descriptors(detector_extractor1.descriptors,detector_extractor2.descriptors)

In [35]:
displaying_matches(img1, img2, detector_extractor1.keypoints, detector_extractor2.keypoints, matches)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [2]:
a = 2 
if a>1:
    print "1"
elif a>1.2:
    print "2"
else:
    print "3"

1


In [9]:
content = []
with open("test.txt") as f:
    content = f.readlines()
content = [x.split() for x in content] 

In [14]:
print content[0]

['Richard_Gere/Richard_Gere_0006.jpg', 'Richard_Gere/Richard_Gere_0010.jpg', '1']


In [16]:
root_dir = '.\lfw'
imgpath = os.path.join(root_dir,content[0][0])
print imgpath

.\lfw\Richard_Gere/Richard_Gere_0006.jpg


In [18]:
testimg = cv2.imread(imgpath)

In [19]:
cv2.imshow('test',testimg)
cv2.waitKey(0)

-1

In [21]:
print testimg.shape

(250L, 250L, 3L)
